In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle/

/content/drive/MyDrive/Kaggle


In [ ]:
%ls

kaggle.json  skin-cancer-mnist-ham10000.zip


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 --force        # kaggle.json se zip file lani

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
skin-cancer-mnist-ham10000.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 --unzip --force    # Zip file to unzip

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.20G/5.20G [01:21<00:00, 101MB/s] 
100% 5.20G/5.20G [01:21<00:00, 68.1MB/s]


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load metadata
metadata_path = "/content/drive/MyDrive/Kaggle/HAM10000_metadata.csv"
metadata = pd.read_csv(metadata_path, sep=',')

# Function to load and preprocess images
def load_image(file_path):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    return img

In [ ]:
metadata.columns

Index(['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization'], dtype='object')

In [ ]:
# Prepare dataset
images = []
labels = []

image_folder_1 = '/content/drive/MyDrive/Kaggle/HAM10000_images_part_1'
image_folder_2 = '/content/drive/MyDrive/Kaggle/HAM10000_images_part_2'

for index, row in metadata.iterrows():
    image_id = row['image_id']
    file_path_1 = os.path.join(image_folder_1, f"{image_id}.jpg")
    file_path_2 = os.path.join(image_folder_2, f"{image_id}.jpg")

    if os.path.exists(file_path_1):
        images.append(load_image(file_path_1))
    elif os.path.exists(file_path_2):
        images.append(load_image(file_path_2))
    else:
        print(f"Image {image_id} not found")

    labels.append(row['dx'])

KeyboardInterrupt: 

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Assuming your images are stored in 'train' and 'val' directories
train_generator = train_datagen.flow_from_directory(
    'path/to/train',
    target_size=(150, 150),  # Adjust target size based on your model requirements
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'path/to/val',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
images = np.array(images)
labels = pd.get_dummies(labels).values  # One-hot encoding

# Split dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [ ]:
# Step 5: Define the AlexNet Model
from tensorflow.keras import layers, models

def alexnet_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Conv2D(256, kernel_size=5, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
model = alexnet_model(input_shape=(224, 224, 3), num_classes=labels.shape[1])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=val_generator)


Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 61/251 ━━━━━━━━━━━━━━━━━━━━ 6:10 2s/step - accuracy: 0.5322 - loss: 60.4132